In [ ]:
import sys

sys.path.append("..")

from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.keras_models as Models
from core.evaluation import MLEvaluator
from core.reconstruction import KerasFFRecoBase
import core


PLOTS_DIR = "plots/plot_comp_history_assignment_transformer/"
CONFIG_PATH = "../config/test_load_config.yaml"
import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path, max_events=2_000_000, event_numbers="odd"
)
X_val, y_val = DataProcessor.get_data()

In [ ]:
MODEL_DIRS = [
    "../models/regression_transformer_nu_only/odd_model.keras",
    "../models/regression_transformer_pil/odd_model.keras",
]
MODEL_NAMES = ["RegressionTransformer", "AssignmentTransformer"]

ml_reconstructors = []
for i in range(len(MODEL_DIRS)):
    reconstructor = KerasFFRecoBase(
        data_config, name=MODEL_NAMES[i], perform_regression=False
    )
    reconstructor.load_model(MODEL_DIRS[i])
    ml_reconstructors.append(reconstructor)

evaluator = MLEvaluator(
    reconstructor=ml_reconstructors,
    X_test=X_val,
    y_test=y_val,
)
plt.rcParams.update({"font.size": 14})

In [ ]:
history = ml_reconstructors[0].history.history

In [ ]:
fig, ax = evaluator.plot_training_history()
fig.savefig(PLOTS_DIR + "training_histories_comparison.pdf")